### **LSTM for 1D sequential data**


In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Dense, GlobalMaxPooling1D, LSTM
from tensorflow.keras.models import Model

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu="")
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("all devices:", tf.config.list_logical_devices("TPU"))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
!unzip data.zip

In [ ]:
df = pd.read_csv('data/accelerometer.csv')

In [ ]:
df.head()

In [ ]:
# make sure we have sequential data on the first column and drunkness on the right
x = df[0]
y = df[1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

In [ ]:
D = 16
D

16

In [ ]:
def build_LSTM():
  i = Input((D,))
  x = LSTM(M, return_sequences=True)(i)
  x = GlobalMaxPooling1D()(x)
  x = Dense(1, activation="sigmoid")(x)

  model = Model(i, x)
  return model

In [ ]:
M = 10
with strategy.scope():
  LSTM = build_LSTM()
  LSTM.compile(optimizer="adam",
                loss = "binary_crossentropy",
                metrics = ["accuracy"])
  r = LSTM.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10)

In [ ]:
LSTM.save('LSTM.h5')

LSTM_weights = LSTM.get_weights()
model_dir = "model_dir"
if not os.path.exists(model_dir):
  os.makedirs(model_dir)
np.save(os.path.join(model_dir, 'LSTM_weights'), LSTM_weights)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(LSTM) 
tflite_model = converter.convert()

with open("LSTM.tflite", "wb") as f:
  f.write(tflite_model)

### **ANN version**

In [ ]:
D = 16
def build_ANN():
  i = Input((D,))
  x = Dense(1, activation='sigmoid')(i)

  model = Model(i,x)

  return model

In [ ]:
with strategy.scope():
  ANN = build_ANN()
  ANN.compile(optimizer="adam",
                loss = "binary_crossentropy",
                metrics = ["accuracy"])
  r = ANN.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10)

NameError: ignored

In [ ]:
ANN = build_ANN()
ANN.compile(optimizer="adam",
            loss = "binary_crossentropy",
            metrics = ["accuracy"])

In [ ]:
ANN.save('ANN.h5')

ANN_weights = ANN.get_weights()
model_dir = "model_dir"
if not os.path.exists(model_dir):
  os.makedirs(model_dir)
np.save(os.path.join(model_dir, 'ANN_weights'), ANN_weights)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(ANN) 
tflite_model = converter.convert()

with open("ANN.tflite", "wb") as f:
  f.write(tflite_model)